## 분석에 필요한 library 불러오기

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from datetime import datetime
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import StratifiedKFold

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, f1_score, precision_score, roc_auc_score
from sklearn.metrics import confusion_matrix
from lightgbm import LGBMClassifier

- 구글 드라이브 연동

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


- 경로 지정

In [ ]:
import os
os.chdir("/content/gdrive/MyDrive/2022빅콘테스트_데이터분석리그_데이터분석분야_퓨처스부문_데이터셋_220908")

## 데이터 불러오기

In [ ]:
train = pd.read_csv("./train_2.csv", encoding = 'cp949',  index_col = 0)
test = pd.read_csv("./test_2.csv", encoding = 'cp949',  index_col = 0)

- 파생데이터 파일 불러오기

In [ ]:
count_bank = pd.read_csv("./count_bank.csv", encoding = 'cp949',  index_col = 0)
count_prod = pd.read_csv("./count_prod.csv", encoding = 'cp949',  index_col = 0)

In [ ]:
count_bank.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62 entries, 0 to 61
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   bank_id          62 non-null     int64  
 1   bank_count       62 non-null     int64  
 2   bank_product     62 non-null     int64  
 3   apply_rate_bank  62 non-null     float64
dtypes: float64(1), int64(3)
memory usage: 2.4 KB


In [ ]:
count_prod.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 168 entries, 0 to 167
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   product_id      168 non-null    int64  
 1   product_count   168 non-null    int64  
 2   pro_apply_rate  168 non-null    float64
dtypes: float64(1), int64(2)
memory usage: 5.2 KB


In [ ]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3255194 entries, 0 to 3255084
Data columns (total 28 columns):
 #   Column                               Dtype  
---  ------                               -----  
 0   application_id                       int64  
 1   user_id                              int64  
 2   insert_time                          object 
 3   credit_score                         float64
 4   yearly_income                        float64
 5   income_type                          object 
 6   company_enter_month                  object 
 7   employment_type                      object 
 8   houseown_type                        object 
 9   desired_amount                       float64
 10  purpose                              object 
 11  personal_rehabilitation_yn           float64
 12  personal_rehabilitation_complete_yn  int64  
 13  existing_loan_cnt                    float64
 14  existing_loan_amt                    float64
 15  NA_count                        

- train data + 파생변수 데이터

In [ ]:
train = pd.merge(count_bank, train, on='bank_id', how='left')
train = pd.merge(count_prod, train, on='product_id', how='left')

- test data + 파생변수 데이터

In [ ]:
test = pd.merge(count_bank, test, on='bank_id', how='left')
test = pd.merge(count_prod, test, on='product_id', how='left')

In [ ]:
train.shape

(10263722, 36)

In [ ]:
test.shape

(3242644, 33)

- desired_rate 파생변수 생성

In [ ]:
train['desired_rate'] = round(train['loan_limit']/10,0) / train['desired_amount']
test['desired_rate'] = round(test['loan_limit']/10,0) / test['desired_amount']

In [ ]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3242644 entries, 0 to 3242643
Data columns (total 34 columns):
 #   Column                               Dtype  
---  ------                               -----  
 0   product_id                           int64  
 1   product_count                        int64  
 2   pro_apply_rate                       float64
 3   bank_id                              float64
 4   bank_count                           float64
 5   bank_product                         float64
 6   apply_rate_bank                      float64
 7   application_id                       float64
 8   user_id                              float64
 9   insert_time                          object 
 10  credit_score                         float64
 11  yearly_income                        float64
 12  income_type                          object 
 13  company_enter_month                  object 
 14  employment_type                      object 
 15  houseown_type                   

# 예측 알고리즘 만들기

In [ ]:
train1 = train[['credit_score',  'existing_loan_cnt', 'existing_loan_amt', 'NA_count', 'age', 'company_month',
                'lotloan', 'loan_limit', 'desired_rate', 'loan_rate', 'pre_bank_rate', 'is_applied', 'bank_product', 'apply_rate_bank', 'pro_apply_rate']]

In [ ]:
test1 = test[['credit_score',  'existing_loan_cnt', 'existing_loan_amt', 'NA_count', 'age', 'company_month',
                'lotloan', 'loan_limit', 'desired_rate', 'loan_rate', 'pre_bank_rate', 'bank_product', 'apply_rate_bank', 'pro_apply_rate']]

In [ ]:
test1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10263722 entries, 0 to 10263721
Data columns (total 14 columns):
 #   Column             Dtype  
---  ------             -----  
 0   credit_score       float64
 1   existing_loan_cnt  float64
 2   existing_loan_amt  float64
 3   NA_count           float64
 4   age                float64
 5   company_month      float64
 6   lotloan            float64
 7   loan_limit         float64
 8   desired_rate       float64
 9   loan_rate          float64
 10  pre_bank_rate      float64
 11  bank_product       int64  
 12  apply_rate_bank    float64
 13  pro_apply_rate     float64
dtypes: float64(13), int64(1)
memory usage: 1.4 GB


In [ ]:
train1['lotloan'] = train1['lotloan'].astype('category')
train1['lotloan'] = train1['lotloan'].astype('category')

test1['lotloan'] = test1['lotloan'].astype('category')
test1['lotloan'] = test1['lotloan'].astype('category')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

- StratifiedKFold 교차검증하기

In [ ]:
x = train1.drop(['is_applied'], axis=1)
y = train1['is_applied']

In [ ]:
skf = StratifiedKFold(n_splits=5)

for train_idx, test_idx in skf.split(x, y):
  X_train = x.iloc[train_idx]
  X_test = x.iloc[test_idx]
  y_train, y_test = y[train_idx], y[test_idx]


  model = LGBMClassifier(n_estimators=300)
  lgb_eval =[(X_test, y_test)]
  model.fit(X_train,y_train,
                eval_set=lgb_eval,
                eval_metric='logloss',
                verbose=True,
                early_stopping_rounds=100)

  preds = model.predict(X_test)
  print(f1_score(y_test, preds))

[1]	valid_0's binary_logloss: 0.200426	valid_0's binary_logloss: 0.200426
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's binary_logloss: 0.193735	valid_0's binary_logloss: 0.193735
[3]	valid_0's binary_logloss: 0.189364	valid_0's binary_logloss: 0.189364
[4]	valid_0's binary_logloss: 0.185321	valid_0's binary_logloss: 0.185321
[5]	valid_0's binary_logloss: 0.182339	valid_0's binary_logloss: 0.182339
[6]	valid_0's binary_logloss: 0.180319	valid_0's binary_logloss: 0.180319
[7]	valid_0's binary_logloss: 0.178881	valid_0's binary_logloss: 0.178881
[8]	valid_0's binary_logloss: 0.17731	valid_0's binary_logloss: 0.17731
[9]	valid_0's binary_logloss: 0.175427	valid_0's binary_logloss: 0.175427
[10]	valid_0's binary_logloss: 0.174066	valid_0's binary_logloss: 0.174066
[11]	valid_0's binary_logloss: 0.173072	valid_0's binary_logloss: 0.173072
[12]	valid_0's binary_logloss: 0.171845	valid_0's binary_logloss: 0.171845
[13]	valid_0's binary_logloss: 0.170684	valid_0'

In [ ]:
y_pro = pd.DataFrame(model.predict_proba(test1))

In [ ]:
y_pro

,0,1
0,0.938931,0.061069
1,0.948999,0.051001
2,0.941070,0.058930
3,0.948999,0.051001
4,0.948565,0.051435
...,...,...
3242639,0.948565,0.051435
3242640,0.950470,0.049530
3242641,0.950470,0.049530
3242642,0.950470,0.049530


In [ ]:
train['is_applied'].sum()/train['is_applied'].count()

0.05400555471007496

In [ ]:
y_pro['is_applied'] = 0
y_pro.loc[y_pro[1]>0.1, 'is_applied']=1

In [ ]:
y_pro

,0,1,is_applied
0,0.938931,0.061069,0
1,0.948999,0.051001,0
2,0.941070,0.058930,0
3,0.948999,0.051001,0
4,0.948565,0.051435,0
...,...,...,...
3242639,0.948565,0.051435,0
3242640,0.950470,0.049530,0
3242641,0.950470,0.049530,0
3242642,0.950470,0.049530,0


In [ ]:
test['is_applied']=y_pro['is_applied']

In [ ]:
# 답안 파일 불러오기
file = pd.read_csv("/content/gdrive/MyDrive/데이터분석분야_퓨처스부문_평가데이터.csv")
file.drop('is_applied', axis=1, inplace=True)

In [ ]:
file.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3255194 entries, 0 to 3255193
Data columns (total 3 columns):
 #   Column          Dtype  
---  ------          -----  
 0   application_id  int64  
 1   product_id      int64  
 2   is_applied      float64
dtypes: float64(1), int64(2)
memory usage: 74.5 MB


In [ ]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3242644 entries, 0 to 3242643
Data columns (total 34 columns):
 #   Column                               Dtype  
---  ------                               -----  
 0   product_id                           int64  
 1   product_count                        int64  
 2   pro_apply_rate                       float64
 3   bank_id                              float64
 4   bank_count                           float64
 5   bank_product                         float64
 6   apply_rate_bank                      float64
 7   application_id                       float64
 8   user_id                              float64
 9   insert_time                          object 
 10  credit_score                         float64
 11  yearly_income                        float64
 12  income_type                          object 
 13  company_enter_month                  object 
 14  employment_type                      object 
 15  houseown_type                   

- train data 파생변수 합치기

In [ ]:
com2 =  pd.merge(file, test[['application_id','product_id', 'is_applied']], on=['application_id','product_id'], how='left')

In [ ]:
com2

,application_id,product_id,is_applied
0,4,220,0.0
1,4,191,0.0
2,8,29,1.0
3,8,159,0.0
4,8,85,1.0
...,...,...,...
3255189,2167778,258,0.0
3255190,2167791,29,0.0
3255191,2167822,149,0.0
3255192,2167822,157,0.0


In [ ]:
com2

,application_id,product_id,is_applied
0,4,220,0.0
1,4,191,0.0
2,8,29,1.0
3,8,159,0.0
4,8,85,1.0
...,...,...,...
3255189,2167778,258,0.0
3255190,2167791,29,0.0
3255191,2167822,149,0.0
3255192,2167822,157,0.0


In [ ]:
com2.loc[com2['is_applied'].isna(), 'is_applied']=0

In [ ]:
com2.isna().sum()

application_id    0
product_id        0
is_applied        0
dtype: int64

In [ ]:
com2.to_csv('./결과제출.csv')